In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("python spark sql basic example") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

# Chapter 6. 다양한 데이터 타입 다루기
## 6.1 API는 어디서 찾을까?
- DataFrame 메서드
  - DataFrameStatFunctions : 다양한 통계적 함수를 제공
  - DataFrameNaFunctions : null 데이터를 다루는데 필요한 함수를 제공
- Column 메서드
  - alias, contain과 같은 컬럼과 관련된 여러가지 메서드를 제공
  - org.apache.spark.sql.function 패키지는 데이터 타입과 관련된 다양한 함수를 제공

In [3]:
# DataFrame 생성
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("./FileStore/tables/data/retail-data/by-day/2010_12_01-ec65d.csv")

df.printSchema()
df.createOrReplaceTempView("dfTable")
df.show(5)

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: timestamp (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|2010-12-01 08:26:00| 2.75| 17850.0|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|2010-12-01 08:26:00| 3.39| 17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

## 6.2 스파크 데이터 타입으로 변환하기
- lit 함수 : 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 벼환
- sparkSQL에서는 스파크 데이터 타입으로 변경할 수 없음

In [5]:
# 스파크 데이터 타입으로 변환
from pyspark.sql.functions import lit

df.select(lit(5), lit("five"), lit(5.0))

Out[97]: DataFrame[5: int, five: string, 5.0: double]

## 6.3 불리언 데이터 타입 다루기

In [7]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365) \
  .select("InvoiceNo", "Description") \
  .show(5, False)

df.where("InvoiceNo = 536365") \
  .show(5, False)

df.where("InvoiceNo <> 536365") \
  .show(5, False)

+---------+-----------------------------+
InvoiceNo|Description |
+---------+-----------------------------+
536366 |HAND WARMER UNION JACK |
536366 |HAND WARMER RED POLKA DOT |
536367 |ASSORTED COLOUR BIRD ORNAMENT|
536367 |POPPY'S PLAYHOUSE BEDROOM |
536367 |POPPY'S PLAYHOUSE KITCHEN |
+---------+-----------------------------+
only showing top 5 rows

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode|Description |Quantity|InvoiceDate |UnitPrice|CustomerID|Country |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
536365 |85123A |WHITE HANGING HEART T-LIGHT HOLDER |6 |2010-12-01 08:26:00|2.55 |17850.0 |United Kingdom|
536365 |71053 |WHITE METAL LANTERN |6 |2010-12-01 08:26:00|3.39 |17850.0 |United Kingdom|
536365 |84406B |CREAM CUPID HEARTS COAT HANGER |8 |2010-12-01 08:26:00|2.75 |17850.0 |United Kingdom|
536365 |84029G |KNITTED UNION FLAG HOT WATER BOTTLE|6 |2010-12-01 08:26:00|3.39 |17850.0 |United Kingdom|
536365 |84029E |RED WOOLLY HOTTIE WHITE HEART. |6 |2010-12-01 08:26:00|3.39 |17850.0 |United Kingdom|
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode|Description |Quantity|InvoiceDate |UnitPrice|CustomerID|Country |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
536366 |22633 |HAND WARMER UNION JACK |6 |2010-12-01 08:28:00|1.85 |17850.0 |United Kingdom|
536366 |22632 |HAND WARMER RED POLKA DOT |6 |2010-12-01 08:28:00|1.85 |17850.0 |United Kingdom|
536367 |84879 |ASSORTED COLOUR BIRD ORNAMENT|32 |2010-12-01 08:34:00|1.69 |13047.0 |United Kingdom|
536367 |22745 |POPPY'S PLAYHOUSE BEDROOM |6 |2010-12-01 08:34:00|2.1 |13047.0 |United Kingdom|
536367 |22748 |POPPY'S PLAYHOUSE KITCHEN |6 |2010-12-01 08:34:00|2.1 |13047.0 |United Kingdom|
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

In [8]:
""" an, or 사용한 불리언 표현식 """

from pyspark.sql.functions import instr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1 # Locate the position of the first occurrence of substr column in the given string.
                                                      # Returns null if either of the arguments are null.
display(df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536544,DOT,DOTCOM POSTAGE,1,2010-12-01T14:32:00.000+0000,569.77,null,United Kingdom
536592,DOT,DOTCOM POSTAGE,1,2010-12-01T17:06:00.000+0000,607.49,null,United Kingdom


In [9]:
""" instr 함수 """

df.withColumn("added", instr(df.Description, "POSTAGE")).where("added > 1").show() # 8번째 글자에 'POSTAGE'가 시작됨

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|added|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
 536544| DOT|DOTCOM POSTAGE| 1|2010-12-01 14:32:00| 569.77| null|United Kingdom| 8|
 536592| DOT|DOTCOM POSTAGE| 1|2010-12-01 17:06:00| 607.49| null|United Kingdom| 8|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+

In [10]:
# 불리언 컬럼

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") > 1 # 'POSTAGE' 글자가 첫번째가 아닌 경우
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)) \
  .where("isExpensive") \
  .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
unitPrice|isExpensive|
+---------+-----------+
 569.77| true|
 607.49| true|
+---------+-----------+

In [11]:
from pyspark.sql.functions import expr, col # 파이썬은 not이 존재하지 않음

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")) \
  .where("isExpensive") \
  .select("Description", "UnitPrice").show(5)

+--------------+---------+
 Description|UnitPrice|
+--------------+---------+
DOTCOM POSTAGE| 569.77|
DOTCOM POSTAGE| 607.49|
+--------------+---------+

## 6.4 수치형 데이터 타입 다루기

In [13]:
from pyspark.sql.functions import expr, pow

fabricateQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), fabricateQuantity.alias("realQuantity")).show(5)

+----------+------------------+
CustomerID| realQuantity|
+----------+------------------+
 17850.0|239.08999999999997|
 17850.0| 418.7156|
 17850.0| 489.0|
 17850.0| 418.7156|
 17850.0| 418.7156|
+----------+------------------+
only showing top 5 rows

In [14]:
# SQL문으로 위와 동일한 결과를 실행해보자
df.selectExpr(
  "CustomerID",
  "(POWER(Quantity * UnitPrice, 2.0) + 5) as realQuantity"
).show(5)

+----------+------------------+
CustomerID| realQuantity|
+----------+------------------+
 17850.0|239.08999999999997|
 17850.0| 418.7156|
 17850.0| 489.0|
 17850.0| 418.7156|
 17850.0| 418.7156|
+----------+------------------+
only showing top 5 rows

In [15]:
# 반올림하는 round 함수
from pyspark.sql.functions import lit, round, bround # round(반올림), bround(반내림)

df.select(round(lit("2.5"), 1), bround(lit("2.5"), 1)).show(2) # 1차원

+-------------+--------------+
round(2.5, 1)|bround(2.5, 1)|
+-------------+--------------+
 2.5| 2.5|
 2.5| 2.5|
+-------------+--------------+
only showing top 2 rows

In [16]:
# 피어슨 상관계수 계싼
from pyspark.sql.functions import corr

df.stat.corr("Quantity", "UnitPrice")

Out[106]: -0.04112314436835551

In [17]:
# 기술통계 계산
df.describe().show()
df.describe("InvoiceNo").show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
summary| InvoiceNo| StockCode| Description| Quantity| UnitPrice| CustomerID| Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
 count| 3108| 3108| 3098| 3108| 3108| 1968| 3108|
 mean| 536516.684944841|27834.304044117645| null| 8.627413127413128| 4.151946589446603|15661.388719512195| null|
 stddev|72.89447869788873|17407.897548583845| null|26.371821677029203|15.638659854603892|1854.4496996893627| null|
 min| 536365| 10002| 4 PURPLE FLOCK D...| -24| 0.0| 12431.0| Australia|
 max| C536548| POST|ZINC WILLIE WINKI...| 600| 607.49| 18229.0|United Kingdom|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+

+-------+-----------------+
summary| InvoiceNo|
+-------+-----------------+
 count| 3108|
 mean| 536516.684944841|
 stddev|72.89447869788873|
 min| 536365|
 max| C536548|
+-------+-----------------+

In [18]:
# 백분위수를 구하는 approxQuantile
quantileProbs = [0.5]
relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError)
# :relError: The relative target precision to achieve
#   (>= 0). If set to zero, the exact quantiles are computed, which
#   could be very expensive. Note that values greater than 1 are
#   accepted but give the same result as 1.

Out[108]: [2.51]

In [19]:
# 교차표를 생성하는 crosstab
df.select("StockCode").distinct().show() # row name
df.select("Quantity").distinct().show() # column name
display(df.stat.crosstab("StockCode", "Quantity")) # pivot

StockCode_Quantity,-1,-10,-12,-2,-24,-3,-4,-5,-6,-7,1,10,100,11,12,120,128,13,14,144,15,16,17,18,19,192,2,20,200,21,216,22,23,24,25,252,27,28,288,3,30,32,33,34,36,384,4,40,432,47,48,480,5,50,56,6,60,600,64,7,70,72,8,80,9,96
22578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21327,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22064,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21080,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22219,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21908,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22818,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
15056BL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
72817,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22545,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# 고유 아이디를 생성하는 monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(10)

+-----------------------------+
monotonically_increasing_id()|
+-----------------------------+
 0|
 1|
 2|
 3|
 4|
 5|
 6|
 7|
 8|
 9|
+-----------------------------+
only showing top 10 rows

## 6.5 문자열 데이터 타입 다루기
- initcap
- lower, upper
- llit, ltrim, rtrim, rpad, lpad, trim
- 정규표현식
  - regexp_extract, regexp_replace
  - translate : 문자 치환
  - regexp_extract : 단어 추출
  - instr : 단어 존재 유무 확인
  - locate : 인수의 개수가 동적으로 변하는 상황과 문자열의 위치를 찾는 locate함수

In [22]:
# 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경
# initcapital
from pyspark.sql.functions import initcap

df.select(initcap(col("Description"))).show(2, False)

+----------------------------------+
initcap(Description) |
+----------------------------------+
White Hanging Heart T-light Holder|
White Metal Lantern |
+----------------------------------+
only showing top 2 rows

In [23]:
from pyspark.sql.functions import lower, upper

df.select(col("Description"), lower(col("Description")), upper(col("Description"))).show(5)

+--------------------+--------------------+--------------------+
 Description| lower(Description)| upper(Description)|
+--------------------+--------------------+--------------------+
WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
 WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
CREAM CUPID HEART...|cream cupid heart...|CREAM CUPID HEART...|
KNITTED UNION FLA...|knitted union fla...|KNITTED UNION FLA...|
RED WOOLLY HOTTIE...|red woolly hottie...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

In [24]:
# 문자열 주변의 공백을 제거
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

df.select(
    ltrim(lit("   HELLO   ")).alias("ltrim"),
    rtrim(lit("   HELLO   ")).alias("rtrim"),
    trim(lit("   HELLO   ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")
).show(2)

+--------+--------+-----+---+----------+
 ltrim| rtrim| trim| lp| rp|
+--------+--------+-----+---+----------+
HELLO | HELLO|HELLO|HEL|HELLO |
HELLO | HELLO|HELLO|HEL|HELLO |
+--------+--------+-----+---+----------+
only showing top 2 rows

### 6.5.1 정규 표현식
- 존재 여부를 확인하거나 일치하는 모든 문자열을 치환
- 정규 표현식을 위해 regexp_extract 함수와 regexp_reaplace함수를 사용

In [26]:
from pyspark.sql.functions import regexp_replace

# 단어 치환
regex_string = "BLACK|WHITE|GREEN|BLUE"
df.select(
  regexp_replace(col("Description"), regex_string, "COLOR").alias("color_replace"),
  col("Description")
).show(5)

+--------------------+--------------------+
 color_replace| Description|
+--------------------+--------------------+
COLOR HANGING HEA...|WHITE HANGING HEA...|
 COLOR METAL LANTERN| WHITE METAL LANTERN|
CREAM CUPID HEART...|CREAM CUPID HEART...|
KNITTED UNION FLA...|KNITTED UNION FLA...|
RED WOOLLY HOTTIE...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+
only showing top 5 rows

pyspark.sql.functions.translate
```python
SELECT translate('AaBbCc', 'abc', '123');

> A1B2C3
```

In [28]:
# 문자 치환, translate
from pyspark.sql.functions import translate

df.select(translate(col("Description"), "LEET", "12"),
         col("Description")).show(5)

+--------------------------------+--------------------+
translate(Description, LEET, 12)| Description|
+--------------------------------+--------------------+
 WHI2 HANGING H2AR...|WHITE HANGING HEA...|
 WHI2 M2A1 1AN2RN| WHITE METAL LANTERN|
 CR2AM CUPID H2ARS...|CREAM CUPID HEART...|
 KNI2D UNION F1AG ...|KNITTED UNION FLA...|
 R2D WOO11Y HOI2 W...|RED WOOLLY HOTTIE...|
+--------------------------------+--------------------+
only showing top 5 rows

In [29]:
# 단어 추출, regexp_extract
from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|GREEN|BLUE)"
df.select(
  regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
  col("Description")
).show(5)

+-----------+--------------------+
color_clean| Description|
+-----------+--------------------+
 WHITE|WHITE HANGING HEA...|
 WHITE| WHITE METAL LANTERN|
 |CREAM CUPID HEART...|
 |KNITTED UNION FLA...|
 WHITE|RED WOOLLY HOTTIE...|
+-----------+--------------------+
only showing top 5 rows

In [30]:
# 단어 존재유무, contain
from pyspark.sql.functions import instr

containBlack = instr(col("Description"), "BLACK") > 1
containWhite = instr(col("Description"), "WHITE") > 1

# where("hasSimpleColor")는 True만 리턴
df.withColumn("hasSimpleColor", containBlack | containWhite) \
    .where("hasSimpleColor") \
    .select("Description").show(3, False) # show()에서 False함수는 20이상 확인

+----------------------------------+
Description |
+----------------------------------+
RED WOOLLY HOTTIE WHITE HEART. |
WOOD 2 DRAWER CABINET WHITE FINISH|
WOOD S/3 CABINET ANT WHITE FINISH |
+----------------------------------+
only showing top 3 rows

- spark.sql.functions.locate

```python
l = [('sdiblue', 1), ("bicke", 3), ("nible", 4)]
df1 = spark.createDataFrame(l, ['s', 'k'])
df1.select(locate('b', df1.s, 1).alias('s')).collect()

> [Row(s=4), Row(s=1), Row(s=3)]
```

In [32]:
# 인수의 개수가 동적으로 변하는 상황과 문자열의 위치를 찾는 locate함수
from pyspark.sql.functions import expr, locate

simple_color = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
  return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string) # color_strings 단어가 시작되는 문자기준(단어기준X) 위치

selected_cols = [color_locator(df.Description, c) for c in simple_color]
selected_cols

Out[118]: [Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>]

In [33]:
selected_cols.append(expr("*")) # column 타입이어야 함
selected_cols

display(df.select(*selected_cols).limit(5))

is_black,is_white,is_red,is_green,is_blue,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
false,true,false,false,false,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01T08:26:00.000+0000,2.55,17850.0,United Kingdom
false,true,false,false,false,536365,71053,WHITE METAL LANTERN,6,2010-12-01T08:26:00.000+0000,3.39,17850.0,United Kingdom
false,false,false,false,false,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01T08:26:00.000+0000,2.75,17850.0,United Kingdom
false,false,false,false,false,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01T08:26:00.000+0000,3.39,17850.0,United Kingdom
false,true,true,false,false,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01T08:26:00.000+0000,3.39,17850.0,United Kingdom


In [34]:
display(df.select(*selected_cols).where(expr("is_white OR is_red")) \
  .select(col("Description")).limit(5))

Description
WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
RED WOOLLY HOTTIE WHITE HEART.
HAND WARMER RED POLKA DOT
RED COAT RACK PARIS FASHION


In [35]:
selected_cols

Out[121]: [Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>,
 Column<b'unresolvedstar()'>]

## 6.6 날짜와 타임스탬프 데이터 타입 다루기
- 스파크든 2가지 시간 정보만 다룸
  - 날짜 정보만 가지는 date
  - 날짜와 시간 정보를 모두 갖는 timestamp
- 시간 설정이 필요하면 스파크 SQL설저의 spark.conf.sessionLocalTimeZone 속성으로 핸들링 가능
  - 자바 TimeZone 포맷을 따라야 함
- TimestampType 클래스는 초 단위 정밀도만 지원
  - 초 단위 이상 정밀도 요구 시 long 데이터타입으로 데이터를 변환해 처리하는 우회 정책이 필요함

In [37]:
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10) \
      .withColumn("today", current_date()) \
      .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("datatable")
dateDF.printSchema()

dateDF.show(3, False)

root
-- id: long (nullable = false)
-- today: date (nullable = false)
-- now: timestamp (nullable = false)

+---+----------+-----------------------+
id |today |now |
+---+----------+-----------------------+
0 |2020-05-30|2020-05-30 14:44:09.371|
1 |2020-05-30|2020-05-30 14:44:09.371|
2 |2020-05-30|2020-05-30 14:44:09.371|
+---+----------+-----------------------+
only showing top 3 rows

In [38]:
# 날짜를 더하거나 빽;
from pyspark.sql.functions import date_sub, date_add

dateDF.select(
  expr("today"),
  date_sub(col("today"), 5),
  date_add(expr("today"), 5)
).show(1)

+----------+------------------+------------------+
 today|date_sub(today, 5)|date_add(today, 5)|
+----------+------------------+------------------+
2020-05-30| 2020-05-25| 2020-06-04|
+----------+------------------+------------------+
only showing top 1 row

In [39]:
# 두 날짜 사이의 일/개월 수를 파악
from pyspark.sql.functions import datediff, months_between, to_date

# 현재 날짜에서 7일 제외 후 datediff결과 확인
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
  .select(datediff(col("week_ago"), col("today"))).show(1)


# 개월 수 차이 파악
dateDF.select(
  to_date(lit("2016-01-01")).alias("start"),
  to_date(lit("2017-05-22")).alias("end"))\
  .select(months_between(col("start"), col("end"))).show(1)

+-------------------------+
datediff(week_ago, today)|
+-------------------------+
 -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
months_between(start, end, true)|
+--------------------------------+
 -16.67741935|
+--------------------------------+
only showing top 1 row

In [40]:
# 문자열을 날짜로 변환
# 자바의 simpleDateFormat 클래스가 지원하는 포맷 사용 필요
from pyspark.sql.functions import to_date, lit

spark.range(5).withColumn("date", lit("2017-01-01"))\
  .select(to_date(col("date"))).show()

+---------------+
to_date(`date`)|
+---------------+
 2017-01-01|
 2017-01-01|
 2017-01-01|
 2017-01-01|
 2017-01-01|
+---------------+

In [41]:
# 파싱오류로 날짜가 null로 반환되는 사례
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show()

+---------------------+---------------------+
to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
 null| 2017-12-11|
+---------------------+---------------------+

In [42]:
# SimpleDateFormat 표준을 활용해 날짜 포맷을 지정
from pyspark.sql.functions import to_date

dateFormat = 'yyyy-dd-MM'

cleanDateDF = spark.range(1).select(
  to_date(lit("2017-12-11"), dateFormat).alias('date'),
  to_date(lit("2017-20-12"), dateFormat).alias("date2")
)

cleanDateDF.createOrReplaceTempView("dateTable2")

In [43]:
spark.sql("""
SELECT * FROM dateTable2
""").show()

+----------+----------+
 date| date2|
+----------+----------+
2017-11-12|2017-12-20|
+----------+----------+

In [44]:
# 항상 날짜 포맷을 지정해야 하는 to_timestamp 함수
from pyspark.sql.functions import to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
 2017-11-12 00:00:00|
+----------------------------------+

In [45]:
# 날짜 비교
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
 date| date2|
+----------+----------+
2017-11-12|2017-12-20|
+----------+----------+

## 6.7 null값 다루기
- null값을 사용하는 것보다 명시적으로 사용하는 것이 항상 좋음
- null값을 허용하지 않는 컬럼을 선언해도 강제성은 없음
- nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 돕는 역할
- null값을 다루는 방법은 2가지
  - 명시적으로 null을 제거
  - 전역 또는 컬럼 단위로 null 값을 특정 값으로 채움

### 6.7.1 coalesce

In [48]:
from pyspark.sql.functions import coalesce

df.select(coalesce(col("Description"), col("CustomerID"))).show()

+---------------------------------+
coalesce(Description, CustomerID)|
+---------------------------------+
 WHITE HANGING HEA...|
 WHITE METAL LANTERN|
 CREAM CUPID HEART...|
 KNITTED UNION FLA...|
 RED WOOLLY HOTTIE...|
 SET 7 BABUSHKA NE...|
 GLASS STAR FROSTE...|
 HAND WARMER UNION...|
 HAND WARMER RED P...|
 ASSORTED COLOUR B...|
 POPPY'S PLAYHOUSE...|
 POPPY'S PLAYHOUSE...|
 FELTCRAFT PRINCES...|
 IVORY KNITTED MUG...|
 BOX OF 6 ASSORTED...|
 BOX OF VINTAGE JI...|
 BOX OF VINTAGE AL...|
 HOME BUILDING BLO...|
 LOVE BUILDING BLO...|
 RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows

### 6.7.2 ifnull, nullif, nvl, nvl2
- SQL 함수이며 DataFrame의 select 표혀식으로 사용 가능
  - ifnull(null, 'return_value') : 두 번째 값을, 아니라면 첫번쨰 값을 반환
  - nullif('value', 'value') : 두 값이 같으면 null
  - nvl(null, 'return_value') : 두 번째 값을, 아니라면 첫 번째 값을 반환
  - nvl2('not_null', 'return_value', 'else_value') : 두 번째 값을, 아니라면 세번째 값을 반환

In [50]:
display(spark.sql("""
SELECT
    ifnull(null, 'return_value'),
    nullif('value', 'value'),
    nvl(null, 'return_value'),
    nvl2('not null', 'return_value', 'else_value')
"""))

"ifnull(NULL, 'return_value')","nullif('value', 'value')","nvl(NULL, 'return_value')","nvl2('not null', 'return_value', 'else_value')"
return_value,null,return_value,return_value


### 6.7.3 drop
- null 값을 가진 로우를 제거

In [52]:
df.na.drop()
df.na.drop('any').show(1) # 로우 컬럼 값 중에서 하나라도 null이면 제거
df.na.drop('all').show(1) # 로우 컬럼 값 중에서 모두 null이면 제거

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

In [53]:
# 배열 형태의 컬럼을 인수로 전달 가능
df.na.drop("all", subset = ("StockCode", "InvoiceNo")).show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|2010-12-01 08:26:00| 2.55| 17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

### 6.7.4 fill
- fill : null을 특정한 값으로 채움

In [55]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

myManualSchema = StructType([
  StructField("string_null", StringType(), True),
  StructField("string2_null", StringType(), True),
  StructField("number_null", DoubleType(), True)
])

myRows = []
myRows.append(Row("Hello", None, float(5))) # string 컬럼에 null 포함
myRows.append(Row(None, 'World', None))

myDF = spark.createDataFrame(myRows, myManualSchema)
myDF.show()

+-----------+------------+-----------+
string_null|string2_null|number_null|
+-----------+------------+-----------+
 Hello| null| 5.0|
 null| World| null|
+-----------+------------+-----------+

- fill 함수는 DataType이 동일한 컬럼의 null만 치환
- 숫자형 또한 치환할 값의 DataType이 동일해야 함

In [57]:
myDF.na.fill("All null values become this string").show() # 동일한 string타입 컬럼의 null만 치환하는 것을 볼 수 있음
myDF.na.fill(5.0).show() # 숫자형 또한 동일한 데이터타입만 치환하는 것을 볼 수 있음

+--------------------+--------------------+-----------+
 string_null| string2_null|number_null|
+--------------------+--------------------+-----------+
 Hello|All null values b...| 5.0|
All null values b...| World| null|
+--------------------+--------------------+-----------+

+-----------+------------+-----------+
string_null|string2_null|number_null|
+-----------+------------+-----------+
 Hello| null| 5.0|
 null| World| 5.0|
+-----------+------------+-----------+

In [58]:
# 딕셔너리 타입을 사용해서 다수의 컬럼에 fill메서드를 적용
fill_cols_vals = {'number_null' : 5.0, "string_null" : "no_value"}
myDF.na.fill(fill_cols_vals).show()

+-----------+------------+-----------+
string_null|string2_null|number_null|
+-----------+------------+-----------+
 Hello| null| 5.0|
 no_value| World| 5.0|
+-----------+------------+-----------+

### 6.7.5 replace

In [60]:
# 조건에 따라 다른 값으로 대체
myDF.show()
myDF.na.replace([""], ["Hello"], "string_null").show()

+-----------+------------+-----------+
string_null|string2_null|number_null|
+-----------+------------+-----------+
 Hello| null| 5.0|
 null| World| null|
+-----------+------------+-----------+

+-----------+------------+-----------+
string_null|string2_null|number_null|
+-----------+------------+-----------+
 Hello| null| 5.0|
 null| World| null|
+-----------+------------+-----------+

## 6.8 정렬하기
- asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last

## 6.9 복합 데이터 다루기
- 구조체, 배열, 맵

### 6.9.1 구조체
- DataFrame 내부의 DataFrame
- 다수의 컬럼을 괄호로 묶어 생성 가능
- 문법에 점(.)을 사용하거나 getField 메서드를 사용
- (*\) 문자로 모든 값을 조회할 수 있음

In [63]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.show(5, False)

+---------------------------------------------+
complex |
+---------------------------------------------+
[WHITE HANGING HEART T-LIGHT HOLDER, 536365] |
[WHITE METAL LANTERN, 536365] |
[CREAM CUPID HEARTS COAT HANGER, 536365] |
[KNITTED UNION FLAG HOT WATER BOTTLE, 536365]|
[RED WOOLLY HOTTIE WHITE HEART., 536365] |
+---------------------------------------------+
only showing top 5 rows

In [64]:
# 아래의 연산은 모두 동일
complexDF.select("complex.Description", "complex.InvoiceNo")
complexDF.select(col("complex").getField("Description"), col("complex").getField("InvoiceNo"))
complexDF.select("complex.*")
complexDF.select(col("complex.*"))
complexDF.selectExpr("complex.*").show(5)

+--------------------+---------+
 Description|InvoiceNo|
+--------------------+---------+
WHITE HANGING HEA...| 536365|
 WHITE METAL LANTERN| 536365|
CREAM CUPID HEART...| 536365|
KNITTED UNION FLA...| 536365|
RED WOOLLY HOTTIE...| 536365|
+--------------------+---------+
only showing top 5 rows

### 6.9.2 배열
- 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환

### Split

In [67]:
from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

+---------------------+
split(Description, )|
+---------------------+
 [WHITE, HANGING, ...|
 [WHITE, METAL, LA...|
+---------------------+
only showing top 2 rows

In [68]:
""" 배열값의 조회 """
df.select(split(col("Description"), " ").alias("array_col"))\
    .selectExpr("array_col[0]", "array_col[1]", "array_col[2]").show(2)

+------------+------------+------------+
array_col[0]|array_col[1]|array_col[2]|
+------------+------------+------------+
 WHITE| HANGING| HEART|
 WHITE| METAL| LANTERN|
+------------+------------+------------+
only showing top 2 rows

### 배열의 길이

In [70]:
# 배열의 길이 : size 함수
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

+---------------------------+
size(split(Description, ))|
+---------------------------+
 5|
 3|
+---------------------------+
only showing top 2 rows

### array_contains
- array_contains 함수를 사용해 배열에 특정 값이 존재하는지 확인

In [72]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(10)

+--------------------------------------------+
array_contains(split(Description, ), WHITE)|
+--------------------------------------------+
 true|
 true|
 false|
 false|
 true|
 false|
 false|
 false|
 false|
 false|
+--------------------------------------------+
only showing top 10 rows

### explode
- 배열 타입의 컬럼을 입력받고 컬럼의 배열값에 포함된 모든 값을 로우로 변환

In [74]:
from pyspark.sql.functions import split, explode

explodedDF = df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded")

print(df.select("Description").count())
print(explodedDF.select("exploded").count())

3108
14414

In [75]:
explodedDF.select("Description", "exploded").count() # 큰 쪽으로 카운드

Out[146]: 14414

In [76]:
explodedDF.select("Description", "exploded").take(10) # Description 컬럼이 Group이 되어 중복됨

Out[147]: [Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='WHITE'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HANGING'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HEART'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='T-LIGHT'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HOLDER'),
 Row(Description='WHITE METAL LANTERN', exploded='WHITE'),
 Row(Description='WHITE METAL LANTERN', exploded='METAL'),
 Row(Description='WHITE METAL LANTERN', exploded='LANTERN'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CREAM'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CUPID')]

### 6.9.3 맵
- map 함수와 컬럼의 키 값 쌍을 이용해 생성
- 적합한 키를 사용해 데이터를 조회할 수 있으며 해당키가 없다면 null값을 반환함

In [78]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).take(2)

Out[148]: [Row(complex_map={'WHITE HANGING HEART T-LIGHT HOLDER': '536365'}),
 Row(complex_map={'WHITE METAL LANTERN': '536365'})]

In [79]:
# 맵의 데이터 조회
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(10)

+--------------------------------+
complex_map[WHITE METAL LANTERN]|
+--------------------------------+
 null|
 536365|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
+--------------------------------+
only showing top 10 rows

In [80]:
# 맵 분해
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")) \
  .selectExpr("explode(complex_map)").show(2)

+--------------------+------+
 key| value|
+--------------------+------+
WHITE HANGING HEA...|536365|
 WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows

## 6.10 JSON 다루기

In [82]:
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString
""")

jsonDF.show(1, False)

+-------------------------------------------+
jsonString |
+-------------------------------------------+
{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+

In [83]:
# 인라인 쿼리로 JSON 조회하기
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
  get_json_object(col("jsonString"), "jsonString.myJSONKey.myJSONValue[1]").alias("column"),
  json_tuple(col("jsonString"), "myJSONKey")
).show(1, False)

+------+-----------------------+
column|c0 |
+------+-----------------------+
null |{"myJSONValue":[1,2,3]}|
+------+-----------------------+

In [84]:
# StructType을 Json 문자열로 변경
from pyspark.sql.functions import to_json

df.selectExpr("(InvoiceNo, Description) as myStruct") \
  .select(to_json(col("myStruct"))).take(3)

Out[159]: [Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}'),
 Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}'),
 Row(structstojson(myStruct)='{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}')]

In [85]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType([
  StructField("InvoiceNo", StringType(), True),
  StructField("Description", StringType(), True),
])

df.selectExpr("(InvoiceNo, Description) as myStruct") \
  .select(to_json(col("myStruct")).alias("newJSON")) \
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2) # 키를 컬럼명으로 값을 로우로 변경

+----------------------+--------------------+
jsontostructs(newJSON)| newJSON|
+----------------------+--------------------+
 [536365, WHITE HA...|{"InvoiceNo":"536...|
 [536365, WHITE ME...|{"InvoiceNo":"536...|
+----------------------+--------------------+
only showing top 2 rows

## 6.11 사용자 정의 함수
- User defined function(UDF)는 레포트별로 데이터를 처리하는 함수이며, SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록됨
- 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하 발생
- 언어별로 성능차이가 존재, 파이썬에서도 사용할 수 있으므로 자바나 스칼라도 함수 작성을 추천함

In [87]:
udfExDF = spark.range(5).toDF("num")

def power3(double_value):
  return double_value ** 3

power3(2.0)

Out[163]: 8.0

In [88]:
from pyspark.sql.functions import udf

power3udf = udf(power3)

udfExDF.select(power3udf(col("num"))).show(3)

+-----------+
power3(num)|
+-----------+
 0|
 1|
 8|
+-----------+
only showing top 3 rows